In [4]:
# 1 imports
import wfdb, matplotlib.pyplot as plt, numpy as np
from scipy.signal import butter, filtfilt #FOR preprocessing



#print(wfdb.__version__)
# 2 load one record
record = wfdb.rdrecord(r"C:\Users\oussk\OneDrive\Desktop\02 y 02 s\Advanced Logic Design\ecg-project\data\raw\101")
ann= wfdb.rdann(r"C:\Users\oussk\OneDrive\Desktop\02 y 02 s\Advanced Logic Design\ecg-project\data\raw\101", 'atr')

length=2 #PERSONAL CHOICE TO BE JUSTIFIED
signal = record.p_signal[:,0] #channel 0
fs = record.fs               #sampling rate
segment = signal[0:fs*length]

#store sym (N/A) in the array for each window
labels_in_window = [sym for samp, sym in zip(ann.sample, ann.symbol) if 0 <= samp < length*fs]

#here we check if all are N so normal, and if at least 1 is not N we go abnormal
#since + is for beat change we can ignore it and focus on N A V L.. only
valid_beats = [sym for sym in labels_in_window if sym not in ['+']]
label = "abnormal" if any(sym != "N" for sym in valid_beats) else "normal"


print("Annotations in window:", labels_in_window)
print("Segment shape:", segment.shape[0])
print("Label:", label)

def bandpass_filter(signal, fs, low=0.5, high=40, order=4):
    nyq = 0.5 * fs
    b, a = butter(order, [low/nyq, high/nyq], btype='band')
    return filtfilt(b, a, signal)

def preprocess_segment(segment, fs):
    # 1. Filter
    segment_filt = bandpass_filter(segment, fs)
    # 2. Normalize (z-score) USED FOR CNN
    segment_norm_zscore = (segment_filt - np.mean(segment_filt)) / np.std(segment_filt)
    # or (z-score) USED FOR fpga fixed-point
    segment_norm_minmax = 2 * (segment_filt - np.min(segment_filt)) / (np.max(segment_filt) - np.min(segment_filt)) - 1

    return segment_norm_zscore

segment_norm=preprocess_segment(segment,fs)
print(segment_norm)

Annotations in window: ['+', 'N', 'N', 'N']
Segment shape: 720
Label: normal
[-3.86922666e-02 -4.55466101e-02 -5.14072249e-02 -5.48507006e-02
 -5.40746888e-02 -4.74841023e-02 -3.46095879e-02 -1.67727090e-02
  3.11097223e-03  2.17210853e-02  3.68717732e-02  4.83397122e-02
  5.73868070e-02  6.54583885e-02  7.32348197e-02  8.08106084e-02
  8.85965236e-02  9.77263776e-02  1.09194212e-01  1.22354796e-01
  1.34437625e-01  1.42217214e-01  1.45233291e-01  1.48438012e-01
  1.62122606e-01  1.98455728e-01  2.65943103e-01  3.64423123e-01
  4.83134724e-01  6.02994032e-01  7.02323536e-01  7.63878187e-01
  7.80527052e-01  7.57321509e-01  7.08927327e-01  6.53403982e-01
  6.05178802e-01  5.70477677e-01  5.47026137e-01  5.27481034e-01
  5.04241279e-01  4.72937877e-01  4.33072275e-01  3.86226651e-01
  3.33769454e-01  2.75975596e-01  2.13035429e-01  1.46774857e-01
  8.13623466e-02  2.21815628e-02 -2.63699586e-02 -6.24600828e-02
 -8.66544555e-02 -1.00484105e-01 -1.05359115e-01 -1.02949302e-01
 -9.64875327e